In [1]:
# Generate some sample data to use with the PC algorithm

N = 1000 # number of data points

# define simple linear model with added noise
x = randn(N)
v = x + randn(N)*0.25
w = x + randn(N)*0.25
z = v + w + randn(N)*0.25
s = z + randn(N)*0.25

df = (x=x, v=v, w=w, z=z, s=s)

(x = [1.849399785088361, 1.0805343163783885, -0.6304925413613067, -0.6920644688186596, 0.270406943790535, 1.872286932542604, -0.3718031186803632, 0.13460254512686753, 0.34370306515477417, -0.11414787529876254  …  1.0447152733147553, 0.45294267914611597, 0.6784823039405918, 0.7057752093866282, 0.5181587260772056, 0.5815211392764982, 0.0030123586705395255, -0.37285916147211906, 0.20585696090605443, -0.22135878781363902], v = [1.978310300899033, 0.789257467303089, -0.44147343840964237, -0.6697392309512689, 0.009821959002910441, 2.1301231153414077, -0.2866196065488114, 0.06354484914539779, 0.13562749058057386, -0.053636296587986645  …  0.9837466731906284, 0.34392495191057887, 0.26460073159273595, 0.4413105589937123, 0.7614970238092088, 0.32024653558167937, 0.19621707419587298, -0.3634888869615789, 0.18025862076698704, -0.5568201639273278], w = [1.5316282203147467, 1.342205877288502, -0.3706225167937369, -0.5464913833508198, -0.15606782682184261, 2.1325547210274083, -0.4422755431733666, -0.

In [5]:
import Pkg

Pkg.add("CausalInference")

    Updating registry at `C:\Users\Roberto\.julia\registries\General.toml`
   Resolving package versions...
   Installed TikzGraphs ────── v1.3.0
   Installed LittleCMS_jll ─── v2.12.0+0
   Installed Tectonic ──────── v0.8.0
   Installed Poppler_jll ───── v0.87.0+2
   Installed OpenJpeg_jll ──── v2.4.0+0
   Installed TikzPictures ──── v3.4.2
   Installed CausalInference ─ v0.8.0
    Updating `C:\Users\Roberto\.julia\environments\v1.7\Project.toml`
  [8e462317] + CausalInference v0.8.0
    Updating `C:\Users\Roberto\.julia\environments\v1.7\Manifest.toml`
  [8e462317] + CausalInference v0.8.0
  [9ac5f52a] + Tectonic v0.8.0
  [b4f28e30] + TikzGraphs v1.3.0
  [37f6aa50] + TikzPictures v3.4.2
  [d3a379c0] + LittleCMS_jll v2.12.0+0
  [643b3616] + OpenJpeg_jll v2.4.0+0
  [9c32591e] + Poppler_jll v0.87.0+2
Precompiling project...
  ✓ Tectonic
  ✓ LittleCMS_jll
  ✓ OpenJpeg_jll
  ✓ Poppler_jll
  ✓ TikzPictures
  ✓ TikzGraphs
  ✓ CausalInference
  7 dependencies successfully precompiled in 10 s

In [6]:
est_g = pcalg(df, 0.01, gausscitest)

LoadError: UndefVarError: pcalg not defined

In [7]:
plot_pc_dag(est_g, [String(k) for k in keys(df)])

LoadError: UndefVarError: plot_pc_dag not defined